In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

top_5000_coca_word_frequencyのダウンロードサイト:
https://www.wordfrequency.info/samples.asp
元のファイルは.xlsxだけど、処理したあと、.csvに変換させた(excelファイルのsheet 1の部分の列のデータだけ保留)

In [47]:
df = pd.read_csv('top_5000_coca_word_frequency.csv', encoding='utf-8')
df

,rank,lemma,PoS,perMil
0,1,the,a,50385.16
1,2,be,v,32622.71
2,3,and,c,24952.20
3,4,a,a,24395.69
4,5,of,i,23321.89
...,...,...,...,...
5045,5046,keyboard,n,11.97
5046,5047,persist,v,11.96
5047,5048,wheat,n,11.96
5048,5049,predator,n,11.96


lemmaは同じだけど、PoSが違うため、top 5000で複数回出場した単語はdf_duplicatesとunique_duplicatesでまとめた

In [48]:
value_counts = df['lemma'].value_counts().reset_index()
value_counts.columns = ['lemma', 'duplicate_count']

duplicate_values = value_counts[value_counts['duplicate_count'] > 1]['lemma']

df_duplicates = df[df['lemma'].isin(duplicate_values)].copy()
df_duplicates['duplicate_count'] = df_duplicates['lemma'].map(
    value_counts.set_index('lemma')['duplicate_count']
)

df_duplicates

,rank,lemma,PoS,perMil,duplicate_count
5,6,to,t,16887.99,2
6,7,in,i,15780.80,3
9,10,it,p,11119.63,2
11,12,to,i,9297.44,2
12,13,that,c,8377.97,3
...,...,...,...,...,...
4990,4991,try,n,12.18,2
4999,5000,sudden,r,12.16,2
5001,5002,japanese,n,12.15,2
5005,5006,associate,j,12.14,3


In [49]:
unique_duplicates = df_duplicates[['lemma', 'duplicate_count']].drop_duplicates('lemma')
unique_duplicates = unique_duplicates.reset_index()
unique_duplicates = unique_duplicates.drop(columns = ['index'])
unique_duplicates

,lemma,duplicate_count
0,to,2
1,in,3
2,it,2
3,that,3
4,for,2
...,...,...
618,seal,2
619,interior,2
620,transport,2
621,cure,2


In [50]:
unique_duplicates_descending = unique_duplicates.sort_values(by='duplicate_count', ascending=False)
unique_duplicates_descending = unique_duplicates_descending.reset_index().drop(columns=['index'])
unique_duplicates_descending

,lemma,duplicate_count
0,like,6
1,no,5
2,top,4
3,past,4
4,good,3
...,...,...
618,seal,2
619,interior,2
620,transport,2
621,cure,2


In [59]:
value_counts_1 = df['lemma'].value_counts().reset_index()
value_counts_1.columns = ['lemma', 'duplicate_count']

duplicate_values_1 = value_counts_1['lemma']

df_extension = df[df['lemma'].isin(duplicate_values_1)].copy()
df_extension['duplicate_count'] = df_extension['lemma'].map(
    value_counts.set_index('lemma')['duplicate_count']
)

df_extension[2105:2109]

,rank,lemma,PoS,perMil,duplicate_count
2105,2106,rise,n,41.90,2
2106,2107,FALSE,j,41.23,1
2107,2108,likely,r,41.83,2
2108,2109,creation,n,41.81,1


In [69]:
unique_freq_ranking = df_extension.groupby('lemma').agg({
    'rank': 'first',
    'PoS': 'first',
    'perMil': 'sum',
    'duplicate_count': 'first'
}).reset_index()

unique_freq_ranking = unique_freq_ranking[['rank', 'lemma', 'PoS', 'perMil', 'duplicate_count']]
unique_freq_ranking = unique_freq_ranking.drop(columns=['PoS', 'rank']).sort_values(by='perMil', ascending=False).reset_index().drop(columns=['index'])
unique_freq_ranking['rank'] = range(1, len(unique_freq_ranking) + 1)
unique_freq_ranking = unique_freq_ranking[['rank', 'lemma', 'perMil', 'duplicate_count']]
unique_freq_ranking

,rank,lemma,perMil,duplicate_count
0,1,the,50385.16,1
1,2,be,32622.71,1
2,3,to,26185.43,2
3,4,and,24952.20,1
4,5,a,24395.69,1
...,...,...,...,...
4375,4376,predator,11.96,1
4376,4377,persist,11.96,1
4377,4378,bizarre,11.96,1
4378,4379,poorly,11.71,1


In [70]:
unique_freq_ranking.to_csv('unique_freq_ranking.csv', index=False)